In [164]:
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64 #Both size for training.
epochs = 100 # Number of epochs to train for.
latent_dim = 256 # Latent dimensionality of the encoding space.
num_samples = 10000 # Number of samples to train on
# Path to the data txt file on disk.
data_path = "D:/Anaconda_all/Deep_New/fra-eng/fra.txt"

In [165]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[:min(num_samples, len(lines) - 1)]:
    input_text , target_text, _ = line.split('\t')
    #We use "tab" as the "start sequences" character
    # for the targets , and "\n" as "end sequences" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [167]:
len(target_texts)

10000

In [141]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [142]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 92
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [143]:
input_token_index = dict(
[(char, i) for i, char in enumerate(input_characters)])

target_token_index = dict(
[(char, i) for i, char in enumerate(target_characters) ])

In [144]:
encoder_input_data = np.zeros(
(len(input_texts), max_encoder_seq_length, num_encoder_tokens), 
    dtype='float32')

decoder_input_data = np.zeros(
(len(input_texts), max_decoder_seq_length, num_decoder_tokens), 
    dtype='float32')

decoder_target_data = np.zeros(
(len(input_texts), max_decoder_seq_length, num_decoder_tokens), 
    dtype='float32')

In [145]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t+1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1
        if t > 0:
            # decode_target_data will be ahead by one timestep.
            # and will not include the start character.
            decoder_target_data[i, t-1, target_token_index[char]] = 1.
    decoder_input_data[i, t+1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
    

In [146]:
encoder_input_data[0].shape,decoder_input_data[0].shape,decoder_target_data[0].shape

((15, 71), (59, 92), (59, 92))

In [147]:
# Define an input sequnce and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_output, state_h, state_c = encoder(encoder_inputs)
# We discard encoder_outputs and only keep the states.
encoder_states = [state_h, state_c]

In [148]:
# Set up the decoder, using encoder_states as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well, we don't use the
# return states in the training model, but we will use them in inference
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

decoder_outputs, _, _, = decoder_lstm(decoder_inputs,
                                    initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [149]:
encoder_input_data.shape,decoder_input_data.shape,decoder_target_data.shape

((10000, 15, 71), (10000, 59, 92), (10000, 59, 92))

In [150]:
encoder_inputs.shape, decoder_inputs.shape, decoder_output.shape

(TensorShape([None, None, 71]),
 TensorShape([None, None, 92]),
 TensorShape([None, None, 92]))

In [151]:
# Define the model that will turn 
# encoder_input_data and decoder_input_data into decoder_target_data
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
             metrics=['accuracy'])


In [152]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
         batch_size=batch_size,
         epochs=epochs,
         validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 74s 496ms/step - loss: 1.1368 - accuracy: 0.7359 - val_loss: 1.0082 - val_accuracy: 0.7273
Epoch 2/100
125/125 [==============================] - 61s 492ms/step - loss: 0.8059 - accuracy: 0.7817 - val_loss: 0.8286 - val_accuracy: 0.7706
Epoch 3/100
125/125 [==============================] - 62s 494ms/step - loss: 0.6658 - accuracy: 0.8151 - val_loss: 0.6917 - val_accuracy: 0.8036
Epoch 4/100
125/125 [==============================] - 62s 499ms/step - loss: 0.5793 - accuracy: 0.8317 - val_loss: 0.6351 - val_accuracy: 0.8127
Epoch 5/100
125/125 [==============================] - 69s 549ms/step - loss: 0.5306 - accuracy: 0.8446 - val_loss: 0.5925 - val_accuracy: 0.8264
Epoch 6/100
125/125 [==============================] - 64s 513ms/step - loss: 0.4938 - accuracy: 0.8549 - val_loss: 0.5572 - val_accuracy: 0.8386
Epoch 7/100
125/125 [==============================] - 64s 515ms/step - loss: 0.4641 - accuracy: 0.8630 - val_loss: 0.5367 -

Epoch 57/100
125/125 [==============================] - 61s 490ms/step - loss: 0.0945 - accuracy: 0.9701 - val_loss: 0.5950 - val_accuracy: 0.8723
Epoch 58/100
125/125 [==============================] - 63s 501ms/step - loss: 0.0925 - accuracy: 0.9704 - val_loss: 0.5952 - val_accuracy: 0.8720
Epoch 59/100
125/125 [==============================] - 61s 492ms/step - loss: 0.0905 - accuracy: 0.9713 - val_loss: 0.6033 - val_accuracy: 0.8714
Epoch 60/100
125/125 [==============================] - 63s 505ms/step - loss: 0.0887 - accuracy: 0.9718 - val_loss: 0.6108 - val_accuracy: 0.8711
Epoch 61/100
125/125 [==============================] - 62s 495ms/step - loss: 0.0870 - accuracy: 0.9721 - val_loss: 0.6085 - val_accuracy: 0.8724
Epoch 62/100
125/125 [==============================] - 62s 495ms/step - loss: 0.0852 - accuracy: 0.9727 - val_loss: 0.6131 - val_accuracy: 0.8717
Epoch 63/100
125/125 [==============================] - 63s 508ms/step - loss: 0.0836 - accuracy: 0.9733 - val_loss: 0

In [160]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [161]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [163]:
for seq_index in range(100):
    # Take one-sequence (part of training set )
    # for trying out decoding
    input_seq = encoder_input_data[seq_index: seq_index+1]
    decoded_sentences = decode_sequence(input_seq)
    print('-')
    print("Input sentences:",input_texts[seq_index])
    print("Decoded sentences:", decoded_sentences)

-
Input sentences: Go.
Decoded sentences: Marche.

-
Input sentences: Go.
Decoded sentences: Marche.

-
Input sentences: Go.
Decoded sentences: Marche.

-
Input sentences: Hi.
Decoded sentences: Salut.

-
Input sentences: Hi.
Decoded sentences: Salut.

-
Input sentences: Run!
Decoded sentences: Cours !

-
Input sentences: Run!
Decoded sentences: Cours !

-
Input sentences: Run!
Decoded sentences: Cours !

-
Input sentences: Run!
Decoded sentences: Cours !

-
Input sentences: Run!
Decoded sentences: Cours !

-
Input sentences: Run!
Decoded sentences: Cours !

-
Input sentences: Run!
Decoded sentences: Cours !

-
Input sentences: Run!
Decoded sentences: Cours !

-
Input sentences: Run.
Decoded sentences: Fuyez !

-
Input sentences: Run.
Decoded sentences: Fuyez !

-
Input sentences: Run.
Decoded sentences: Fuyez !

-
Input sentences: Run.
Decoded sentences: Fuyez !

-
Input sentences: Run.
Decoded sentences: Fuyez !

-
Input sentences: Run.
Decoded sentences: Fuyez !

-
Input sentences: 